In [1]:
import praw
import os
import pandas as pd
import numpy as np
import markovify
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from nltk.tokenize import word_tokenize
import nltk

In [3]:
reddit = praw.Reddit(
    client_id = os.getenv("CLIENT_ID"),
    client_secret = os.getenv("CLIENT_SECRET"),
    user_agent= os.getenv("USER_AGENT")
)

In [4]:
print(reddit.read_only)

True


In [5]:
for submission in reddit.subreddit("AmITheAsshole").hot(limit=10):
    print(submission.title)

New Resources for Anyone Looking to Help Those in an Unhealthy or Abusive Relationships
Monthly Open Forum December 2020
AITA for not helping my former “stepmom” and step-siblings after my dads death?
AITA for refusing to role-play as Moana and crushing a little girl’s dreams?
AITA for convincing my girlfriend NOT to wear a white dress on our wedding day?
AITA for setting a "pee trap" for my wife?
AITA for not including my dead niece in my pregnancy announcement?
AITA for not giving my son the ps4 we bought him for Christmas?
AITA for telling a guy behind me in the grocery store line to mind his fucking business?
AITA for not wanting to forgive my brother and for making my mom sad at Christmas?


In [7]:
titles = open("title_aita.txt", "a")

for submission in reddit.subreddit("AITAFiltered").new(limit=2000):
    titles.write(submission.title)
    titles.write("\n")

In [112]:
len(flaired_titles)

3722

In [9]:
flaired_titles = []

for submission in reddit.subreddit("AmITheAsshole").new(limit=2000):
    item = {"title": submission.title, "judgment": submission.link_flair_text}
    flaired_titles.append(item)
    
for submission in reddit.subreddit("AmITheAsshole").hot(limit=2000):
    item = {"title": submission.title, "judgment": submission.link_flair_text}
    flaired_titles.append(item)

for submission in reddit.subreddit("AmITheAsshole").top(limit=2000):
    item = {"title": submission.title, "judgment": submission.link_flair_text}
    flaired_titles.append(item)
    
for submission in reddit.subreddit("AITAFiltered").new(limit=2000):
    item = {"title": submission.title, "judgment": submission.link_flair_text}
    flaired_titles.append(item)
    
df = pd.DataFrame(flaired_titles)

In [114]:
for submission in reddit.subreddit("AmITheAsshole").rising(limit=2000):
    if not submission.link_flair_text == "Not the A-hole":
        item = {"title": submission.title, "judgment": submission.link_flair_text}
        flaired_titles.append(item)
    
for submission in reddit.subreddit("AmITheAsshole").controversial(limit=2000):
    if not submission.link_flair_text == "Not the A-hole":
        item = {"title": submission.title, "judgment": submission.link_flair_text}
        flaired_titles.append(item)
    
for submission in reddit.subreddit("AITAFiltered").rising(limit=2000):
    if not submission.link_flair_text == "Not the A-hole":
        item = {"title": submission.title, "judgment": submission.link_flair_text}
        flaired_titles.append(item)
    
for submission in reddit.subreddit("AITAFiltered").controversial(limit=2000):
    if not submission.link_flair_text == "Not the A-hole":
        item = {"title": submission.title, "judgment": submission.link_flair_text}
        flaired_titles.append(item)

df = pd.DataFrame(flaired_titles)

In [115]:
df = df.dropna()
df = df.drop_duplicates(subset="title")
df

title        judgment
283   AITA for yelling at my aunt to shut up at my m...  Not the A-hole
284          AITA Got mad at mom over possibly sick pet  Not the A-hole
285   AITA For Telling my mom I love my family but h...  Everyone Sucks
286           AITA for calling out my mom on her racism  Not the A-hole
287   AITA for cutting my hair without my mom’s perm...  Not the A-hole
...                                                 ...             ...
4653              AITA for calling my friends fascists?         Asshole
4654           AITA For Lying To My Friend About My Age         Asshole
4655  AITA for eating birthday cake of co-worker I d...         Asshole
4657  AITA for ‘lying’ to get my family to move states?         Asshole
4659  AITA for telling my sister to not want some do...         Asshole

[2612 rows x 2 columns]

In [116]:
t = open("title_aita.txt").read()

In [117]:
title_generator = markovify.Text(t)

In [118]:
tagged_data = [TaggedDocument(words=word_tokenize(_d.lower()), tags=[str(i)]) for i, _d in enumerate(df['title'])]

In [119]:
tagged_data

[TaggedDocument(words=['aita', 'for', 'yelling', 'at', 'my', 'aunt', 'to', 'shut', 'up', 'at', 'my', 'mom', "'s", 'funeral', '?'], tags=['0']),
 TaggedDocument(words=['aita', 'got', 'mad', 'at', 'mom', 'over', 'possibly', 'sick', 'pet'], tags=['1']),
 TaggedDocument(words=['aita', 'for', 'telling', 'my', 'mom', 'i', 'love', 'my', 'family', 'but', 'hate', 'them', 'too', '.'], tags=['2']),
 TaggedDocument(words=['aita', 'for', 'calling', 'out', 'my', 'mom', 'on', 'her', 'racism'], tags=['3']),
 TaggedDocument(words=['aita', 'for', 'cutting', 'my', 'hair', 'without', 'my', 'mom', '’', 's', 'permission', '?'], tags=['4']),
 TaggedDocument(words=['aita', 'for', 'not', 'apologising', 'to', 'my', 'mother', 'because', 'of', 'kitty', 'litter', '.'], tags=['5']),
 TaggedDocument(words=['wibta', 'if', 'i', 'refuse', 'to', 'talk', 'to', 'my', 'dad', 'until', 'he', 'agrees', 'that', 'my', 'long-distance', 'boyfriend', 'is', 'real', '?'], tags=['6']),
 TaggedDocument(words=['wibta', 'if', 'i', 'get'

In [120]:
max_epochs = 100
vec_size = 100
alpha = 0.025

model = Doc2Vec(vector_size=vec_size,
                alpha=alpha, 
                min_alpha=0.00025,
                min_count=1,
                dm=1)

model.build_vocab(tagged_data)

for epoch in range(max_epochs):
    print(f"Iteration {epoch}")
    model.train(tagged_data,
                total_examples=model.corpus_count,
                epochs=model.epochs)
    # decrease the learning rate
    model.alpha -= 0.0002
    # fix the learning rate, no decay
    model.min_alpha = model.alpha

model.save("d2v.model")

Iteration 0
Iteration 1
Iteration 2
Iteration 3
Iteration 4
Iteration 5
Iteration 6
Iteration 7
Iteration 8
Iteration 9
Iteration 10
Iteration 11
Iteration 12
Iteration 13
Iteration 14
Iteration 15
Iteration 16
Iteration 17
Iteration 18
Iteration 19
Iteration 20
Iteration 21
Iteration 22
Iteration 23
Iteration 24
Iteration 25
Iteration 26
Iteration 27
Iteration 28
Iteration 29
Iteration 30
Iteration 31
Iteration 32
Iteration 33
Iteration 34
Iteration 35
Iteration 36
Iteration 37
Iteration 38
Iteration 39
Iteration 40
Iteration 41
Iteration 42
Iteration 43
Iteration 44
Iteration 45
Iteration 46
Iteration 47
Iteration 48
Iteration 49
Iteration 50
Iteration 51
Iteration 52
Iteration 53
Iteration 54
Iteration 55
Iteration 56
Iteration 57
Iteration 58
Iteration 59
Iteration 60
Iteration 61
Iteration 62
Iteration 63
Iteration 64
Iteration 65
Iteration 66
Iteration 67
Iteration 68
Iteration 69
Iteration 70
Iteration 71
Iteration 72
Iteration 73
Iteration 74
Iteration 75
Iteration 76
Iteration

In [121]:
df["vector"] = df["title"].apply(lambda x: np.array(model.infer_vector(x.split(" "))))

In [122]:
list(df["judgment"]).count("Not the A-hole")

1239

In [123]:
from sklearn.model_selection import train_test_split

In [124]:
x_train, x_val, y_train, y_val = train_test_split(df["vector"].to_numpy(), df["judgment"], test_size = .1, random_state=13)

In [125]:
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score

In [126]:
xgb_model = XGBClassifier()

In [127]:
xgb_model.fit(np.stack(x_train),y_train)

/Users/gabrielebranco/.local/share/virtualenvs/redditship-bot-UJrqqXO8/lib/python3.8/site-packages/xgboost/sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[08:29:34] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1057: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.300000012, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=100, n_jobs=4, num_parallel_tree=1,
              objective='multi:softprob', random_state=0, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=None, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [128]:
data = np.array([model.infer_vector("WIBTA if i imposed a reverse study curfew on my sister".split(" "))])
xgb_model.predict(data)

array(['Asshole'], dtype=object)

In [129]:
title_generator.make_sentence()

'AITA if I stopped making/sharing food with my wife put up a Christmas present?'

In [130]:
data = np.array([model.infer_vector("AITA for not wanting to return something I don't buy my nieces first birthday?"
.split(" "))])
xgb_model.predict(data)

array(['Not the A-hole'], dtype=object)

In [131]:
title = title_generator.make_short_sentence(140)
data = np.array([model.infer_vector(title.split(" "))])
print(title)
xgb_model.predict(data)

AITA for leaving for college and not telling my SO stormed off to bed, making me feel skinnier by comparison?


array(['Asshole'], dtype=object)

In [132]:
title = title_generator.make_short_sentence(140)
data = np.array([model.infer_vector(title.split(" "))])
print(title)
xgb_model.predict(data)

AITA for not being grateful?


array(['Not the A-hole'], dtype=object)

In [133]:
title = title_generator.make_short_sentence(140)
data = np.array([model.infer_vector(title.split(" "))])
print(title)
xgb_model.predict(data)

AITA for yelling at my gf any of the flowers from our moms’s old garden for her again?


array(['Asshole'], dtype=object)

In [134]:
title = title_generator.make_short_sentence(140)
data = np.array([model.infer_vector(title.split(" "))])
print(title)
xgb_model.predict(data)

AITA For being upset at a homeless man?


array(['Asshole'], dtype=object)

In [135]:
title = title_generator.make_short_sentence(140)
data = np.array([model.infer_vector(title.split(" "))])
print(title)
xgb_model.predict(data)

WIBTA if I didn't buy me an asshole.


array(['Asshole'], dtype=object)

In [136]:
title = title_generator.make_short_sentence(120)
data = np.array([model.infer_vector(title.split(" "))])
print(title)
xgb_model.predict(data)

AITA for agreeing to watch my niece when she asked me to pay for the damage she caused to my family business?


array(['Asshole'], dtype=object)

In [137]:
title = title_generator.make_short_sentence(140)
data = np.array([model.infer_vector(title.split(" "))])
print(title)
xgb_model.predict(data)


AITA for calling out my mom sad at Christmas?


array(['Not the A-hole'], dtype=object)

In [138]:
import joblib
joblib.dump(xgb_model, "xgb_model.joblib")
joblib.dump(title_generator, "markovify.joblib")


['markovify.joblib']

In [139]:
class Model:
    def __init__(self):
        self.xgb = joblib.load("xgb_model.joblib")
        self.d2v = Doc2Vec.load("d2v.model")
    
    def _vectorize(self, text):
        return np.array([self.d2v.infer_vector(text.split(" "))])
    
    def judge(self, text):
        return self.xgb.predict(self._vectorize(text))[0]
    
class Title(Model):
    def __init__(self):
        super().__init__()
        t = open("title_aita.txt").read()
        self.title_generator = joblib.load("markovify.joblib")
        
    def generate(self):
        title = ""
        while not ((title.lower().count("wibta") == 1 and title.lower().count("aita") == 0) or (title.lower().count("aita") == 1 and title.lower().count("wibta") == 0)):
            title = self.title_generator.make_short_sentence(140)
        return {"text": title, "judgment": self.judge(title)}

In [140]:
t = Title()

In [141]:
for _ in range(100):
    print(t.generate())

{'text': 'AITA for expecting reimbursement for someone wearing my new, expensive dress before I leave a somewhat negative Airbnb review?', 'judgment': 'Asshole'}
{'text': 'WIBTA if I named my baby after my wife to divorce within one year on their own?', 'judgment': 'Not the A-hole'}
{'text': 'AITA for showing my grandparents house when it happened?', 'judgment': 'Not the A-hole'}
{'text': 'AITA for teasing my son after my child?', 'judgment': 'Asshole'}
{'text': 'AITA For expecting my husband Christmas presents?', 'judgment': 'Asshole'}
{'text': 'AITA For refusing to change my daughter’s mom take her on my wife and kids?', 'judgment': 'Asshole'}
{'text': 'AITA for being a thief?', 'judgment': 'Not the A-hole'}
{'text': 'AITA for warning a group of people of various genders?', 'judgment': 'Not the A-hole'}
{'text': 'Aita for insisting to my mom out of bed by 8a during the summer?', 'judgment': 'Not the A-hole'}
{'text': 'WIBTA if I suggested selling our childhood home to my nan’s for Ch

In [142]:
t.generate()

{'text': 'AITA For cancelling my Amazon Prime account cause my brother money for dropping a class in college?',
 'judgment': 'Asshole'}

In [143]:
t.generate()

{'text': 'AITA for not sharing my inheritance with one child and not telling my friend to stop blaming older people for the flight home?',
 'judgment': 'Not the A-hole'}